In [ ]:
!pip3 install mesa
!pip3 install numpy
!pip3 install matplotlib

In [2]:
# La clase `Model` se hace cargo de los atributos a nivel del modelo, maneja los agentes. 
# Cada modelo puede contener múltiples agentes y todos ellos son instancias de la clase `Agent`.
from mesa import Agent, Model 

# Debido a que es posible tener multiples agentes una sola celda elegimos `MultipleGrid`.
from mesa.space import MultiGrid

# Con `SimultaneousActivation` hacemos que todos los agentes se activen de manera simultanea.
from mesa.time import SimultaneousActivation

# Vamos a hacer uso de `DataCollector` para obtener el grid completo cada paso (o generación) y lo usaremos para graficarlo.
from mesa.datacollection import DataCollector

# mathplotlib lo usamos para graficar/visualizar como evoluciona el autómata celular.
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation
plt.rcParams["animation.html"] = "jshtml"
matplotlib.rcParams['animation.embed_limit'] = 2**128

# Definimos los siguientes paquetes para manejar valores númericos.
import numpy as np
import pandas as pd

# Definimos otros paquetes que vamos a usar para medir el tiempo de ejecución de nuestro algoritmo.
import time
import datetime
import math

In [3]:
def get_grid(model):
    '''
    Esta es una función auxiliar que nos permite guardar el grid para cada uno de los agentes.
    param model: El modelo del cual optener el grid.
    return una matriz con la información del grid del agente.
    '''
    grid = np.zeros((model.grid.width, model.grid.height))
    for cell in model.grid.coord_iter():
        cell_content, x, y = cell
        for content in cell_content:
            if isinstance(content, Cleaner):
                grid[x][y] = 2
            elif grid[x][y] == 0:
                grid[x][y] = content.dirty
    return grid

class RandomWalker(Agent):
    """
    Class implementing random walker methods in a generalized manner.
    Not indended to be used on its own, but to inherit its methods to multiple
    other agents.
    """

    grid = None
    x = None
    y = None

    def __init__(self, unique_id, pos, model):
        """
        grid: The MultiGrid object in which the agent lives.
        x: The agent's current x coordinate
        y: The agent's current y coordinate
        """
        super().__init__(unique_id, model)
        self.pos = pos

    def random_move(self):
        """
        Step one cell in any allowable direction.
        """
        # Pick the next cell from the adjacent cells.
        next_moves = self.model.grid.get_neighborhood(self.pos, True, False)
        next_move = self.random.choice(next_moves)
        # Now move:
        self.model.grid.move_agent(self, next_move)

class Cleaner(RandomWalker):
    """
    A cleaner that walks around and cleanes the floor if it detects that is dirty.
    The init is the same as the RandomWalker.
    """
    def __init__(self, unique_id, pos, model):
        super().__init__(unique_id, pos, model)

    def step(self):
        """
        A model step. Move, then eat grass and reproduce.
        """
        moving = True
        this_cell = self.model.grid.get_cell_list_contents([self.pos])
        for content in this_cell:
            print(content)
            if isinstance(content, Floor):
                print("Floor")
                if content.dirty:
                    content.dirty = 0
                    moving = False
        if moving:
            self.random_move()
            

class Floor(Agent):
    """
    A floor tile that it is cleaned by the Cleaner
    """

    def __init__(self, unique_id, pos, model, dirty):
        """
        Creates a floor tile
        Args:
            dirty: (boolean) Whether the floor tile is dirty or not
        """
        super().__init__(unique_id, model)
        self.dirty = dirty
        self.pos = pos
                
            
class GameLifeModel(Model):
    '''
    Define el modelo de la actividad.
    '''
    def __init__(self, width, height, initial_dirty_tiles, initial_cleaners):
        super().__init__()
        self.height = height
        self.width = width
        self.initial_cleaners = initial_cleaners
        self.initial_dirty_tiles = initial_dirty_tiles
        self.grid = MultiGrid(self.width, self.height, True)
        self.schedule = SimultaneousActivation(self)
        
        # Aquí definimos con colector para obtener el grid completo.
        self.datacollector = DataCollector(
            model_reporters={"Grid": get_grid})
        
        # Create cleaners:
        for i in range(self.initial_cleaners):
            x = 1
            y = 1
            cleaner = Cleaner(self.next_id(), (x, y), self)
            self.grid.place_agent(cleaner, (x, y))
            self.schedule.add(cleaner)
            
        # Create dirty and clean tiles
        counter_dirty_tiles = 0
        for agent, x, y in self.grid.coord_iter():
            if counter_dirty_tiles >= self.initial_dirty_tiles:
                return
            dirty = self.random.choice([0, 1])
            if dirty:
                counter_dirty_tiles += 1

            floor_tile = Floor(self.next_id(), (x, y), self, dirty)
            self.grid.place_agent(floor_tile, (x, y))
            self.schedule.add(floor_tile)
        
    
    def step(self):
        '''
        En cada paso el colector tomará la información que se definió y almacenará el grid para luego graficarlo.
        '''
        self.datacollector.collect(self)
        self.schedule.step()

In [4]:
# Definimos el tamaño del Grid
height_m = 10
width_n = 10

# Definimos el número de generaciones a correr
PORCENTAGE_DIRTY_TILES = 50
AMOUNT_DIRTY_TILES = math.floor((height_m * width_n) * (PORCENTAGE_DIRTY_TILES/100))
MAX_TIME = 0.5
NUM_CLEANERS = 3

# Registramos el tiempo de inicio y corremos el modelo
start_time = time.time()
limit_time = datetime.timedelta(seconds=MAX_TIME)
model = GameLifeModel(height_m, width_n, AMOUNT_DIRTY_TILES, NUM_CLEANERS)
while datetime.timedelta(seconds=(time.time() - start_time)) < limit_time:
    model.step()

# Imprimimos el tiempo que le tomó correr al modelo.
print('Tiempo de ejecución:', str(datetime.timedelta(seconds=(time.time() - start_time))))

Cell total content: 
[<__main__.Cleaner object at 0x7f6830159a90>, <__main__.Cleaner object at 0x7f6830159ad0>, <__main__.Cleaner object at 0x7f6830159b10>, <__main__.Floor object at 0x7f6830159d50>]
Each content:
Floor
Cell total content: 
[<__main__.Cleaner object at 0x7f6830159ad0>, <__main__.Cleaner object at 0x7f6830159b10>, <__main__.Floor object at 0x7f6830159d50>]
Each content:
Floor
Cell total content: 
[<__main__.Cleaner object at 0x7f6830159b10>, <__main__.Floor object at 0x7f6830159d50>]
Each content:
Floor
Cell total content: 
[<__main__.Floor object at 0x7f68309795d0>, <__main__.Cleaner object at 0x7f6830159a90>]
Each content:
Floor
Cell total content: 
[<__main__.Floor object at 0x7f68304ac710>, <__main__.Cleaner object at 0x7f6830159ad0>]
Each content:
Floor
Cell total content: 
[<__main__.Floor object at 0x7f6830159b50>, <__main__.Cleaner object at 0x7f6830159b10>]
Each content:
Floor
Cell total content: 
[<__main__.Floor object at 0x7f683013b210>, <__main__.Cleaner ob

Cell total content: 
[<__main__.Cleaner object at 0x7f6830159a90>, <__main__.Cleaner object at 0x7f6830159b10>]
Each content:
Cell total content: 
[<__main__.Cleaner object at 0x7f6830159a90>, <__main__.Cleaner object at 0x7f6830159b10>]
Each content:
Cell total content: 
[<__main__.Floor object at 0x7f6830159ed0>, <__main__.Cleaner object at 0x7f6830159ad0>]
Each content:
Floor
Cell total content: 
[<__main__.Cleaner object at 0x7f6830159a90>, <__main__.Cleaner object at 0x7f6830159b10>]
Each content:
Cell total content: 
[<__main__.Cleaner object at 0x7f6830159a90>, <__main__.Cleaner object at 0x7f6830159b10>]
Each content:
Cell total content: 
[<__main__.Floor object at 0x7f6830159c10>, <__main__.Cleaner object at 0x7f6830159ad0>]
Each content:
Floor
Cell total content: 
[<__main__.Cleaner object at 0x7f6830159a90>, <__main__.Cleaner object at 0x7f6830159b10>]
Each content:
Cell total content: 
[<__main__.Cleaner object at 0x7f6830159a90>, <__main__.Cleaner object at 0x7f6830159b10>

Cell total content: 
[<__main__.Cleaner object at 0x7f6830159a90>, <__main__.Cleaner object at 0x7f6830159b10>]
Each content:
Cell total content: 
Each content:
Cell total content: 
[<__main__.Cleaner object at 0x7f6830159a90>, <__main__.Cleaner object at 0x7f6830159b10>]
Each content:
Cell total content: 
[<__main__.Cleaner object at 0x7f6830159a90>, <__main__.Cleaner object at 0x7f6830159b10>]
Each content:
Cell total content: 
Each content:
Cell total content: 
[<__main__.Cleaner object at 0x7f6830159a90>, <__main__.Cleaner object at 0x7f6830159b10>]
Each content:
Cell total content: 
[<__main__.Cleaner object at 0x7f6830159a90>, <__main__.Cleaner object at 0x7f6830159b10>]
Each content:
Cell total content: 
Each content:
Cell total content: 
[<__main__.Cleaner object at 0x7f6830159a90>, <__main__.Cleaner object at 0x7f6830159b10>]
Each content:
Cell total content: 
[<__main__.Cleaner object at 0x7f6830159a90>, <__main__.Cleaner object at 0x7f6830159b10>]
Each content:
Cell total co

In [5]:
all_grid = model.datacollector.get_model_vars_dataframe()

In [6]:
%%capture

fig, axs = plt.subplots(figsize=(7,7))
axs.set_xticks([])
axs.set_yticks([])
patch = plt.imshow(all_grid.iloc[0][0], cmap='Greys')

def animate(i):
    patch.set_data(all_grid.iloc[i][0])
    
anim = animation.FuncAnimation(fig, animate, frames=len(all_grid))

In [7]:
anim

In [8]:
#all_grid = model.datacollector.get_model_vars_dataframe()
#print (all_grid.tail()['Movimientos'].sum())


KeyError: 'Movimientos'